In [1]:
!pip install --no-deps seqeval[gpu]
!python -m spacy download en_core_web_lg
!pip install pytorch-pretrained-bert
!pip install PyMuPDF
import locale
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=40aadbee4930bd7c8090da2f34c694f55bbabc83872bae3c73a4cfd0934d27f8
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:000:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.1 MB/s eta 0:00:00
   

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#Import Libraries

In [2]:
import random
random.seed(42)

In [3]:
import numpy as np
import pandas as pd

import spacy
from spacy.training import offsets_to_biluo_tags
nlp=spacy.load('en_core_web_lg')

from tqdm import trange
import torch
import torch.nn.functional as F 
from torch.optim import Adam
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from pytorch_pretrained_bert import BertTokenizer,BertConfig
from pytorch_pretrained_bert import BertForTokenClassification,BertAdam

from seqeval.metrics import classification_report,accuracy_score,f1_score
import warnings
warnings.filterwarnings('ignore')

#Text preprocessing tools
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import fitz

#Make each token predict result into softmax mode
from scipy.special import softmax

2025-05-06 23:59:11.046417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746575951.492121      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746575951.614093      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
#Adding '\n' to the default spacy tokenizer
prefixes=[i+'\\n' for i in nlp.Defaults.prefixes]
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [5]:
#Personal custom tags dictionary
entity_dict={
    'Name':'NAME',
    'College Name':'CLG',
    'Degree':'DEG',
    'Graduation Year':'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at':'COMPANY',
    'Designation':'DESIG',
    'Skills':'SKILLS',
    'Location':'LOC',
    'Email Address':'EMAIL'
}

In [6]:
data="/kaggle/input/bertdata/data"

#Load Data

In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm  
import json

In [8]:
#Loading the dataset
data_file_address='/kaggle/input/bertdata/data/Resumes.json'
df=pd.read_json(data_file_address,lines=True)
df.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [9]:
df.iloc[0]['content']

"Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n\

In [10]:
df.iloc[0]['annotation']

[{'label': ['Skills'],
  'points': [{'start': 1295,
    'end': 1621,
    'text': '\n• Programming language: C, C++, Java\n• Oracle PeopleSoft\n• Internet Of Things\n• Machine Learning\n• Database Management System\n• Computer Networks\n• Operating System worked on: Linux, Windows, Mac\n\nNon - Technical Skills\n\n• Honest and Hard-Working\n• Tolerant and Flexible to Different Situations\n• Polite and Calm\n• Team-Player'}]},
 {'label': ['Skills'],
  'points': [{'start': 993,
    'end': 1153,
    'text': 'C (Less than 1 year), Database (Less than 1 year), Database Management (Less than 1 year),\nDatabase Management System (Less than 1 year), Java (Less than 1 year)'}]},
 {'label': ['College Name'],
  'points': [{'start': 939, 'end': 956, 'text': 'Kendriya Vidyalaya'}]},
 {'label': ['College Name'],
  'points': [{'start': 883, 'end': 904, 'text': 'Woodbine modern school'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 856, 'end': 860, 'text': '2017\n'}]},
 {'label': ['College 

In [11]:
print('Shape')
print('===============')
print(df.shape)
print('\nCount')
print('===============')
print(df.count())
print('\nIsnull Count')
print('===============')
print(df.isnull().sum())
print('\nDescribe')
print('===============')
print(df.info())
print('\nColumns')
print('===============')
print(df.columns)

Shape
(220, 3)

Count
content       220
annotation    220
extras          0
dtype: int64

Isnull Count
content         0
annotation      0
extras        220
dtype: int64

Describe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   content     220 non-null    object 
 1   annotation  220 non-null    object 
 2   extras      0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 5.3+ KB
None

Columns
Index(['content', 'annotation', 'extras'], dtype='object')


In [12]:
#Checking for unique values present in 'extras' column
df['extras'].unique()

array([nan])

In [13]:
#Since 'extras' column contains no information we can drop the column
df=df.drop(['extras'],axis=1)
df.head()

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."


# Parser data
- Parser data into document structure

In [14]:
'''
Case 1: if higher[0] <= lower[1], with the SAME ENT
=====================================================
  higher =  (0, 186, 'SKILLS')   -> Current new record
  lower  =  (653, 656, 'SKILLS') -> Last merged list record
  
  higher[0] <= lower[1]     ,  0 <= 656
  lower[2]   = higher[2]    ,  'SKILLS'
  UB         = max (656,186),  -> UB=656
  merged[-1] = previous record

    Case 1.2: if higher[0] <= lower[1], DIFF ENT
    =====================================================
      higher =  (0, 186, 'NAME')   -> Current new record
      lower  =  (653, 656, 'SKILLS') -> Last merged list record
      
      higher[0] <= lower[1]   , 0 <= 646
      lower[2]  != higher[2]  , 'NAME'!='SKILLS'
      lower[1]   > higher[1]  , 656 > 186 
      merged[-1] =  previous record

Case 1: if higher[0] >= lower[1], if current start value > lower end value
===========================================================================
Append into entity
  '''

def mergeIntervals(intervals):
    #Sort based on the tuple start (start,end,label)
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [15]:
# From 'annotation' column, we are extracting the starting index, ending index, entity label
# So that we can convert the content in BILOU format

def get_entities(df):
    
    entities = []
    
    for i in range(len(df)):
        entity = []
    
        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass
    
        entity = mergeIntervals(entity)
        entities.append(entity)
    
    return entities

In [16]:
#Adding an new col for 'entities'
df['entities']=get_entities(df)
df.head()

,content,annotation,entities
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, NAME), (13, 46, DESIG), (49, 58, COMP..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, NAME), (62, 68, LOC), (104, 148, EMAI..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, NAME), (22, 31, LOC), (65, 117, EMAIL..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, NAME), (13, 51, DESIG), (54, 60, COMP..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, NAME), (14, 22, DESIG), (24, 41, COMP..."


In [17]:
df.iloc[0]['entities']

[(0, 12, 'NAME'),
 (13, 46, 'DESIG'),
 (49, 58, 'COMPANY'),
 (60, 69, 'LOC'),
 (95, 146, 'EMAIL'),
 (372, 405, 'DESIG'),
 (407, 416, 'COMPANY'),
 (727, 770, 'DESIG'),
 (771, 814, 'CLG'),
 (856, 861, 'GRADYEAR'),
 (883, 905, 'CLG'),
 (939, 957, 'CLG'),
 (993, 1154, 'SKILLS'),
 (1295, 1622, 'SKILLS')]

In [18]:
'''
https://spacy.io/api/top-level
Returns a list of strings, describing the tags. Each tag will be in the form of either '','0' or {action}-{label},
where action is one of 'B', 'I', 'L', 'U'.
- '-' us used where the entity offsets don't alighn with the tokenization in the Doc obj. The training algo will view as missing values.
- 0 denotes a non-entity token.
- B denotes the beginnning of a multi-token entity.
- I the inside of an entity of 3 or more tokens.
- L the end of an entity of 2 or more tokens.
- U denotes single-token entity.

Example
==============
doc=nlp('I like London') 
entities=[(7,13,'LOC')] 
tags=offsets_to_biluo_tags(doc,entities) #['O', 'O', 'U-LOC'] 
'''
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]
    
        doc = nlp(text)
    
        tag = offsets_to_biluo_tags(doc, entities)
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text is '.' and tmp[1][i] is 'O':
                loc.append(i)
        loc.append(len(doc))
    
        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos
    
        for d in data:
            tag = ['O' if t is '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)
    
    return sentences, tags

In [19]:
sentences, tags = get_train_data(df)
print(len(sentences), len(tags))

779 779


In [20]:
print(sentences[0])
print(tags[0])

[Abhishek, Jha, 
, Application, Development, Associate, -, Accenture, 
, 
, Bengaluru, ,, Karnataka, -, Email, me, on, Indeed, :, indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a, 
, 
, •, To, work, for, an, organization, which, provides, me, the, opportunity, to, improve, my, skills, 
, and, knowledge, for, my, individual, and, company, 's, growth, in, best, possible, ways]
['B-NAME', 'L-NAME', 'O', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'O', 'U-COMPANY', 'O', 'O', 'U-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'L-EMAIL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [21]:
#Find all unique tags
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])

#tag2idx convert tag to idx, is a dict contains (tag,idx)
tag2idx={t:i for i,t in enumerate(tag_vals)}

#idx2tag convert tag to idx, is a dict contains (idx,tag)
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}

In [22]:
print(tag_vals)
print(tag2idx)
print(idx2tag)

{'U-LOC', 'B-EMAIL', 'I-CLG', 'I-DEG', 'X', 'B-LOC', 'I-LOC', 'I-GRADYEAR', 'B-COMPANY', 'O', 'U-GRADYEAR', 'L-DESIG', 'U-SKILLS', 'B-CLG', 'L-CLG', 'I-COMPANY', 'L-NAME', 'I-EMAIL', 'U-COMPANY', '[CLS]', 'L-SKILLS', 'B-DEG', 'I-NAME', '[SEP]', 'B-GRADYEAR', 'I-SKILLS', 'B-DESIG', 'L-DEG', 'L-GRADYEAR', 'B-YOE', 'U-EMAIL', 'B-NAME', 'L-LOC', 'I-YOE', 'U-YOE', 'U-DEG', 'L-COMPANY', 'L-YOE', 'U-DESIG', 'U-CLG', 'B-SKILLS', 'L-EMAIL', 'I-DESIG'}
{'U-LOC': 0, 'B-EMAIL': 1, 'I-CLG': 2, 'I-DEG': 3, 'X': 4, 'B-LOC': 5, 'I-LOC': 6, 'I-GRADYEAR': 7, 'B-COMPANY': 8, 'O': 9, 'U-GRADYEAR': 10, 'L-DESIG': 11, 'U-SKILLS': 12, 'B-CLG': 13, 'L-CLG': 14, 'I-COMPANY': 15, 'L-NAME': 16, 'I-EMAIL': 17, 'U-COMPANY': 18, '[CLS]': 19, 'L-SKILLS': 20, 'B-DEG': 21, 'I-NAME': 22, '[SEP]': 23, 'B-GRADYEAR': 24, 'I-SKILLS': 25, 'B-DESIG': 26, 'L-DEG': 27, 'L-GRADYEAR': 28, 'B-YOE': 29, 'U-EMAIL': 30, 'B-NAME': 31, 'L-LOC': 32, 'I-YOE': 33, 'U-YOE': 34, 'U-DEG': 35, 'L-COMPANY': 36, 'L-YOE': 37, 'U-DESIG': 38, 'U-

# Make training data
- Set GPU environment
- Load tokenizer and tokenize
- Set 3 embedding - Token embedding, Mask word embedding, Segmentation embedding
- Split dataset into train and validate, then send them to DataLoader

In [23]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu=torch.cuda.device_count()
n_gpu

2

In [24]:
#BERT pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

100%|██████████| 213450/213450 [00:00<00:00, 2705107.28B/s]


## Tokenizer Text

- In hunggieface for bert, when come across OOV, will word piece the word.

- We need to adjust the labels base on the tokenize result, “##abc” need to set label "X".

- Need to set "[CLS]" at front and "[SEP]" at the end, as what the paper do, BERT indexer should add [CLS] and [SEP] tokens.

In [25]:
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):
    
        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']
    
        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')  
                
        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')
    
        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)
    
    return tokenized_texts, word_piece_labels

In [26]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)

In [27]:
#Vector representations of the corresponding words from the input
print(tokenized_texts[0])
print(word_piece_labels[0])

['[CLS]', 'A', '##b', '##his', '##he', '##k', 'J', '##ha', 'Application', 'Development', 'Associate', '-', 'A', '##cc', '##ent', '##ure', 'Bengal', '##uru', ',', 'Karnataka', '-', 'Em', '##ail', 'me', 'on', 'Indeed', ':', 'indeed', '.', 'com', '/', 'r', '/', 'A', '##b', '##his', '##he', '##k', '-', 'J', '##ha', '/', '10', '##e', '##7', '##a', '##8', '##c', '##b', '##7', '##32', '##b', '##c', '##43', '##a', '•', 'To', 'work', 'for', 'an', 'organization', 'which', 'provides', 'me', 'the', 'opportunity', 'to', 'improve', 'my', 'skills', 'and', 'knowledge', 'for', 'my', 'individual', 'and', 'company', "'", 's', 'growth', 'in', 'best', 'possible', 'ways', '[SEP]']
['[CLS]', 'B-NAME', 'X', 'X', 'X', 'X', 'L-NAME', 'X', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'O', 'U-COMPANY', 'X', 'X', 'X', 'U-LOC', 'X', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X',

## Set Token Embedding
- Pad or trim the text and label to fit the need for MAX_LEN

In [28]:
MAX_LEN = 512
bs = 4

#Make text tokens into ids
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))
print(input_ids[0])

512
[  101   138  1830 27516  4638  1377   147  2328 22491  3273  9666   118
   138 19515  3452  3313  7756 12328   117 12247   118 18653 11922  1143
  1113 10364   131  5750   119  3254   120   187   120   138  1830 27516
  4638  1377   118   147  2328   120  1275  1162  1559  1161  1604  1665
  1830  1559 17101  1830  1665 25631  1161   794  1706  1250  1111  1126
  2369  1134  2790  1143  1103  3767  1106  4607  1139  4196  1105  3044
  1111  1139  2510  1105  1419   112   188  3213  1107  1436  1936  3242
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     

In [29]:
'''
pad_sequences -> https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
====================
maxlen=512: maximum length of all sequences
padding    ='post': pad after each sequence
truncating ='post': remove values from sequences larger than maxlen at the end of the sequences

convert_tokens_to_ids -> converts a string to a sequence of ids (int)
'''

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels], maxlen=MAX_LEN, value=tag2idx["O"], 
                     padding="post", dtype="long", truncating="post")
print(len(tags[0]))
print(tags[0])

512
[19 31  4  4  4  4 16  4 26 42 11  9 18  4  4  4  0  4  9  9  9  9  4  9
  9  1 17 17  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  4  9  9  9  9  9 23  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  

## Set Mask Word Embeeding
- For fine tune of predict, with token mask is 1, pad token is 0

In [30]:
#if the current value >0, then assign 1, else =0
attention_masks=[[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

## Split data into Train and Validate
- 70% for training, 30% for validation

In [31]:
#train inputs, validation inputs, train tags, validation tags, train masks, validation masks
tr_inputs,val_inputs,tr_tags,val_tags,tr_masks,val_masks=train_test_split(input_ids,tags,attention_masks, random_state=2000, test_size=0.3)

In [32]:
len(tr_inputs),len(val_inputs),len(tr_tags),len(val_tags),len(tr_masks),len(val_masks)

(545, 234, 545, 234, 545, 234)

### Set data into tensor

In [33]:
tr_inputs=torch.tensor(tr_inputs)
val_inputs=torch.tensor(val_inputs)
tr_tags=torch.tensor(tr_tags)
val_tags=torch.tensor(val_tags)
tr_masks=torch.tensor(tr_masks)
val_masks=torch.tensor(val_masks)           

### Put data into Data Loader

In [34]:
'''
TORCH.UTILS.DATA -> https://pytorch.org/docs/stable/data.html

TensorDataset: Dataset wrapping tensors. Each sample will be retrieved by indexing tensors along the first dimension.
RandomSampler: Samples elements randomly
DataLoader   : Python iterable over a dataset

Notes: Only set token embeeding, attention embedding, no segment embedding
'''

train_data=TensorDataset(tr_inputs,tr_masks,tr_tags)
train_sampler=RandomSampler(train_data) 
train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=bs) 

valid_data=TensorDataset(val_inputs,val_masks,val_tags)
valid_sampler=SequentialSampler(valid_data)
valid_dataloader=DataLoader(valid_data,sampler=valid_sampler,batch_size=bs)

# Train Model


In [35]:
#Defining the model
bert_model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))

100%|██████████| 404400730/404400730 [00:09<00:00, 42424269.24B/s]


In [36]:
#Set model to GPU
bert_model.cuda();

### Set Fine-Tuning method
- Manual optimizer

In [37]:
FULL_FINETUNING=True
#If full tuning=True: Fine tuning all the layers
if FULL_FINETUNING:
  param_optimizer=list(bert_model.named_parameters())
  no_decay=['bias','gamma','beta']

  #n=name, p=parameter
  optimizer_grouped_parameters=[
      #Params that not inside no_decay
      {'params':[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],
       'weight_decay_rate':0.01},

       #Params that are inside no_decay
       {'params':[p for n,p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate':0.0}
        ]

#If full tuning=False -> Not full tuning, only fine tune classifier params
else:
  param_optimizer=list(bert_model.named_parameters())
  optimizer_grouped_parameters=[{'params':[p for n,p in param_optimizer]}]

#Optimizer and learning scheduler
optimizer=Adam(optimizer_grouped_parameters,lr=3e-5)

### Fine-Tuning Model

In [38]:
#Set epoch and grad max num
epochs=10
max_grad_norm=1.0

for _ in trange(epochs, desc='Epoch'):
  #Train loop
  bert_model.train()
  tr_loss=0
  nb_tr_examples,nb_tr_steps=0,0

  for step, batch in enumerate(train_dataloader):
    #Add batch to GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
    
    #Forward pass
    loss=bert_model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    #Backward pass
    loss.backward()

    #Track train loss
    tr_loss+=loss.item()

    nb_tr_examples+=b_input_ids.size(0)
    nb_tr_steps+=1

    #Gradient clipping
    torch.nn.utils.clip_grad_norm_(parameters=bert_model.parameters(),max_norm=max_grad_norm)

    #Update parameters
    optimizer.step()
    bert_model.zero_grad()

  #Print train loss per epoch
  print('Train loss: {}'.format(tr_loss/nb_tr_steps))


Epoch:  10%|█         | 1/10 [01:00<09:05, 60.64s/it]

Train loss: 0.7907516077487138


Epoch:  20%|██        | 2/10 [02:05<08:25, 63.17s/it]

Train loss: 0.338385646360634


Epoch:  30%|███       | 3/10 [03:09<07:25, 63.63s/it]

...

Train loss: 0.06869292608632223


Epoch:  90%|█████████ | 9/10 [09:36<01:04, 64.49s/it]

Train loss: 0.05553923775679874


Epoch: 100%|██████████| 10/10 [10:41<00:00, 64.14s/it]

Train loss: 0.04600018234311664


# Save Bert Model

In [54]:
bert_out_address='/kaggle/working/bert_model'

#Save a trained model, configuration and tokenizer
model_to_save=bert_model.module if hasattr(bert_model,'module') else bert_model

output_model_file='/kaggle/working/bert_model/pytorch_model.bin'
output_config_file='/kaggle/working/bert_model/config.json'

#Save model into file
torch.save(model_to_save.state_dict(),output_model_file)

model_to_save.config.to_json_file(output_config_file)

tokenizer.save_vocabulary(bert_out_address)

#Load back the bert model
bert_model=BertForTokenClassification.from_pretrained(bert_out_address, num_labels=len(tag2idx))

bert_model.cuda();

if n_gpu >1:
    bert_model = torch.nn.DataParallel(bert_model)

In [55]:
bert_out_address = '/kaggle/working/bert_model/'

# Load back the BERT model from the working directory
bert_model = BertForTokenClassification.from_pretrained(bert_out_address, num_labels=len(tag2idx))


# Model Evaluation

In [58]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, classification_report
from pytorch_pretrained_bert import BertTokenizer, BertForTokenClassification

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Define tags for BILUO scheme
labels = ['NAME', 'CLG', 'DEG', 'GRADYEAR', 'YOE', 'COMPANY', 'DESIG', 'SKILLS', 'LOC', 'EMAIL']
tags = ['O'] + [f"{prefix}-{label}" for label in labels for prefix in ['B', 'I', 'L', 'U']] + ['[CLS]', '[SEP]', 'X']
tag2idx = {tag: idx for idx, tag in enumerate(tags)}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

# Load and move model to device
bert_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag2idx))
bert_model = bert_model.to(device)

# Set model to evaluation mode
bert_model.eval()

y_true = []
y_pred = []

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Assuming valid_dataloader is defined as in your notebook
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        logits = bert_model(input_ids=input_ids, attention_mask=input_mask)
        logits = logits.detach().cpu().numpy()

    logits = np.argmax(logits, axis=2)

    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()

    for i, mask in enumerate(input_mask):
        for j, m in enumerate(mask):
            if m:  # Only consider valid tokens (non-padded)
                true_label = idx2tag[label_ids[i][j]]
                pred_label = idx2tag[logits[i][j]]
                if true_label not in ['X', '[CLS]', '[SEP]']:
                    y_true.append(true_label)
                    y_pred.append(pred_label)

# Print evaluation results
print("F1 Score: %f" % f1_score(y_true, y_pred, average='weighted'))
print("Accuracy Score: %f" % accuracy_score(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))

F1 Score: 0.004721
Accuracy Score: 0.004262

Classification Report:
              precision    recall  f1-score   support

       B-CLG     0.0000    0.0000    0.0000        12
   B-COMPANY     0.0000    0.0000    0.0000        46
       B-DEG     0.0000    0.0000    0.0000     10760
     B-DESIG     0.3913    0.0055    0.0109      1629
     B-EMAIL     0.0000    0.0000    0.0000         8
  B-GRADYEAR     0.0000    0.0000    0.0000        63
       B-LOC     0.0000    0.0000    0.0000         0
      B-NAME     0.0006    0.0625    0.0011        32
    B-SKILLS     0.0000    0.0000    0.0000         8
       B-YOE     0.0000    0.0000    0.0000        11
       I-CLG     0.0000    0.0000    0.0000         7
   I-COMPANY     0.0000    0.0000    0.0000         5
       I-DEG     0.0000    0.0000    0.0000        47
     I-DESIG     0.0099    0.0164    0.0123       122
     I-EMAIL     0.0000    0.0000    0.0000         8
  I-GRADYEAR     0.0018    0.0192    0.0033        52
       I-LOC 

BERT preprocessing

In [59]:
# JSON formatting functions
import logging
import re
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r',encoding='utf-8') as f:
          lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    #only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']
                        
                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data     

In [60]:
data = trim_entity_spans(convert_dataturks_to_spacy(data_file_address))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

# Inference
- After we trained a model, we can make it into service, sending the new resume then get the prediction.

Process

1) Load model

2) Load tokenizer

3) Set test query (PDF file)

4) Make query into embedding

5) Predict with model

6) Parser result


In [61]:
def getWordnetPos(words):
    tag=pos_tag([words])[0][1][0].upper()
    tag_dict={"J":wordnet.ADJ,
              "N":wordnet.NOUN,
              "V":wordnet.VERB,
              "R":wordnet.ADV
             }
    return tag_dict.get(tag,wordnet.NOUN)

def cv_preprocessing(cv_data):
  #Tokenization
  tokenized_text=word_tokenize(cv_data)

  #Remove stopwords
  stop_words = set(stopwords.words('english'))
  filter_text=[]
  for token in tokenized_text:
    if token not in stop_words:
          filter_text.append(token)

  #POS and lemmatize
  lemmatizer = WordNetLemmatizer()
  lemmatizeResults=[lemmatizer.lemmatize(token,getWordnetPos(token)) for token in filter_text]
  return ' '.join(lemmatizeResults)

def pdftotext(m,preprocessing=False):
  #Open pdf file
  doc=fitz.open(m)

  #Convert pdf to text
  text=''
  for page in doc:
    text+=page.get_text()

  #Remove new line
  text=' '.join(text.split('\n'))

  if preprocessing:
    return cv_preprocessing(text)
  else:
    return text

In [62]:
def bert_predict(cv_data: str):
    # Token id embedding, mask word embedding
    tokenized_texts = []
    temp_token = []

    # Add [CLS] at the front
    temp_token.append('[CLS]')
    token_list = tokenizer.tokenize(cv_data)
    for token in token_list:
        temp_token.append(token)

    # Trim the token to fit the length requirement
    if len(temp_token) > MAX_LEN - 1:
        temp_token = temp_token[:MAX_LEN - 1]

    # Add [SEP] at the end
    temp_token.append('[SEP]')
    tokenized_texts.append(temp_token)

    # Make id embedding  
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                              maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
    attention_masks = [[float(i > 0) for i in ii] for ii in input_ids]
    segment_ids = [[0] * len(input_id) for input_id in input_ids]

    # Make embeddings into torch tensor
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    segment_ids = torch.tensor(segment_ids)

    with torch.no_grad():
        outputs = bert_model(input_ids.cuda(), token_type_ids=None, attention_mask=None)
        logits = outputs[0]

    predict_results = logits.detach().cpu().numpy()
    results_arrays_soft = softmax(predict_results, axis=-1)
    result_array = results_arrays_soft
    result_list = np.argmax(result_array, axis=-1)[0]  # <-- FIX: take first element to get flat array

    # Get token predict tag
    for i, mark in enumerate(attention_masks[0]):
        if mark > 0:
            predicted_index = int(result_list[i])  # <-- FIX: ensure it's an int
            print(f'{temp_token[i]:50} {idx2tag[predicted_index]}')


# Test with one train data

In [65]:
import torch
import numpy as np
from pytorch_pretrained_bert import BertTokenizer, BertForTokenClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Define tags for BILUO scheme
labels = ['NAME', 'CLG', 'DEG', 'GRADYEAR', 'YOE', 'COMPANY', 'DESIG', 'SKILLS', 'LOC', 'EMAIL']
tags = ['O'] + [f"{prefix}-{label}" for label in labels for prefix in ['B', 'I', 'L', 'U']] + ['[CLS]', '[SEP]', 'X']
tag2idx = {tag: idx for idx, tag in enumerate(tags)}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

# Load and move model to device
bert_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag2idx))
bert_model = bert_model.to(device)

def bert_predict(text):
    # Tokenize input text
    tokenized_texts = []
    tokens = ["[CLS]"] + tokenizer.tokenize(text)[:510] + ["[SEP]"]
    tokenized_texts.append(tokens)

    # Convert tokens to input IDs
    input_ids = pad_sequences(
        [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
        maxlen=512,
        dtype="long",
        truncating="post",
        padding="post"
    )

    # Create attention mask
    attention_masks = [[float(i > 0) for i in ii] for ii in input_ids]

    # Convert to PyTorch tensors and move to device
    input_ids = torch.tensor(input_ids).to(device)
    attention_masks = torch.tensor(attention_masks).to(device)

    # Model in evaluation mode
    bert_model.eval()

    # Forward pass
    with torch.no_grad():
        logits = bert_model(input_ids=input_ids, attention_mask=attention_masks)
        logits = logits.detach().cpu().numpy()

    # Get predicted indices
    predicted_indices = np.argmax(logits, axis=2)[0]  # Take first batch (batch_size=1)

    # Align predictions with tokens
    result = []
    for i, mask in enumerate(attention_masks[0]):
        if mask > 0:  # Only consider non-padded tokens
            predicted_tag = idx2tag[predicted_indices[i]]
            result.append((tokens[i], predicted_tag))

    return result

# Test the function (assuming df is your DataFrame)
import pandas as pd
# Example: df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP/BERT/data/Resumes.json', lines=True)
result = bert_predict(df.iloc[0]['content'])
for token, tag in result:
    print(f"{token:<50} {tag}")

[CLS]                                              [CLS]
ab                                                 U-CLG
##his                                              U-CLG
##he                                               U-CLG
##k                                                I-LOC
j                                                  U-DESIG
##ha                                               L-LOC
application                                        B-CLG
development                                        U-DESIG
associate                                          I-EMAIL
-                                                  U-DESIG
accent                                             L-GRADYEAR
##ure                                              B-CLG
bengal                                             L-GRADYEAR
##uru                                              L-EMAIL
,                                                  U-CLG
karnataka                                          B-EMAIL
-        

In [66]:
bert_predict(df.iloc[0]['content'])

[('[CLS]', '[CLS]'),
 ('ab', 'U-CLG'),
 ('##his', 'U-CLG'),
 ('##he', 'U-CLG'),
 ('##k', 'I-LOC'),
 ('j', 'U-DESIG'),
 ('##ha', 'L-LOC'),
 ('application', 'B-CLG'),
 ('development', 'U-DESIG'),
 ('associate', 'I-EMAIL'),
 ('-', 'U-DESIG'),
 ('accent', 'L-GRADYEAR'),
 ('##ure', 'B-CLG'),
 ('bengal', 'L-GRADYEAR'),
 ('##uru', 'L-EMAIL'),
 (',', 'U-CLG'),
 ('karnataka', 'B-EMAIL'),
 ('-', 'U-DESIG'),
 ('email', 'I-EMAIL'),
 ('me', 'I-EMAIL'),
 ('on', 'I-EMAIL'),
 ('indeed', 'I-EMAIL'),
 (':', 'U-DESIG'),
 ('indeed', 'U-CLG'),
 ('.', 'U-CLG'),
 ('com', 'I-EMAIL'),
 ('/', 'I-EMAIL'),
 ('r', 'B-NAME'),
 ('/', 'U-DESIG'),
 ('ab', 'B-YOE'),
 ('##his', 'U-CLG'),
 ('##he', 'L-EMAIL'),
 ('##k', 'I-YOE'),
 ('-', 'U-DESIG'),
 ('j', 'U-YOE'),
 ('##ha', 'L-LOC'),
 ('/', 'U-DESIG'),
 ('10', 'L-EMAIL'),
 ('##e', 'L-EMAIL'),
 ('##7', 'U-EMAIL'),
 ('##a', 'U-EMAIL'),
 ('##8', 'I-NAME'),
 ('##cb', 'U-EMAIL'),
 ('##7', 'I-NAME'),
 ('##32', 'I-NAME'),
 ('##bc', 'I-NAME'),
 ('##43', 'I-COMPANY'),
 ('##a', 'U

In [67]:
bert_predict(data[0][0])

[('[CLS]', '[CLS]'),
 ('ab', 'U-CLG'),
 ('##his', 'U-CLG'),
 ('##he', 'U-CLG'),
 ('##k', 'I-LOC'),
 ('j', 'U-DESIG'),
 ('##ha', 'L-LOC'),
 ('application', 'B-CLG'),
 ('development', 'U-DESIG'),
 ('associate', 'I-EMAIL'),
 ('-', 'U-DESIG'),
 ('accent', 'L-GRADYEAR'),
 ('##ure', 'B-CLG'),
 ('bengal', 'L-GRADYEAR'),
 ('##uru', 'L-EMAIL'),
 (',', 'U-CLG'),
 ('karnataka', 'B-EMAIL'),
 ('-', 'U-DESIG'),
 ('email', 'I-EMAIL'),
 ('me', 'I-EMAIL'),
 ('on', 'I-EMAIL'),
 ('indeed', 'I-EMAIL'),
 (':', 'U-DESIG'),
 ('indeed', 'U-CLG'),
 ('.', 'U-CLG'),
 ('com', 'I-EMAIL'),
 ('/', 'I-EMAIL'),
 ('r', 'B-NAME'),
 ('/', 'U-DESIG'),
 ('ab', 'B-YOE'),
 ('##his', 'U-CLG'),
 ('##he', 'L-EMAIL'),
 ('##k', 'I-YOE'),
 ('-', 'U-DESIG'),
 ('j', 'U-YOE'),
 ('##ha', 'L-LOC'),
 ('/', 'U-DESIG'),
 ('10', 'L-EMAIL'),
 ('##e', 'L-EMAIL'),
 ('##7', 'U-EMAIL'),
 ('##a', 'U-EMAIL'),
 ('##8', 'I-NAME'),
 ('##cb', 'U-EMAIL'),
 ('##7', 'I-NAME'),
 ('##32', 'I-NAME'),
 ('##bc', 'I-NAME'),
 ('##43', 'I-COMPANY'),
 ('##a', 'U

In [68]:
from sklearn.model_selection import train_test_split

# Assuming `data` is a list of (tokens, labels) tuples
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 176
Test set size: 44


In [100]:
import pandas as pd

df = pd.read_json('/kaggle/input/bertdata/data/Resumes.json', lines=True)
print("Columns in DataFrame:", df.columns)
print("Sample row:", df.iloc[0].to_dict())

Columns in DataFrame: Index(['content', 'annotation', 'extras'], dtype='object')
Sample row: {'content': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n1

In [101]:
import spacy
from spacy.training import offsets_to_biluo_tags

nlp = spacy.load('en_core_web_lg')

def get_train_data(df):
    sentences = []
    tags = []
    entity_dict = {
        'Name': 'NAME',
        'College Name': 'CLG',
        'Degree': 'DEG',
        'Graduation Year': 'GRADYEAR',
        'Years of Experience': 'YOE',
        'Companies worked at': 'COMPANY',
        'Designation': 'DESIG',
        'Skills': 'SKILLS',
        'Location': 'LOC',
        'Email Address': 'EMAIL'
    }
    
    for i in range(len(df)):
        text = df['content'][i]
        annotations = df['annotation'][i] if 'annotation' in df.columns else []
        
        # Convert Dataturks annotations to entity spans
        entities = []
        if annotations:
            for ann in annotations:
                if isinstance(ann, dict) and 'points' in ann and 'label' in ann:
                    label = ann['label'][0] if ann['label'] else None
                    if label in entity_dict:
                        for point in ann['points']:
                            start = point['start']
                            end = point['end']
                            entities.append((start, end, entity_dict[label]))
        
        # Generate BILUO tags
        doc = nlp(text)
        try:
            biluo_tags = offsets_to_biluo_tags(doc, entities)
            sentences.append([token.text for token in doc])
            tags.append(biluo_tags)
        except ValueError as e:
            print(f"Skipping row {i} due to error: {e}")
            continue
    
    return sentences, tags

In [102]:
df = pd.read_json('/kaggle/input/bertdata/data/Resumes.json', lines=True)
sentences, tags = get_train_data(df)
tag_vals = set(['X', '[CLS]', '[SEP]'])
for tag_list in tags:
    tag_vals.update(tag_list)
print("Training tags:", tag_vals)
print("Number of training tags:", len(tag_vals))

tag2idx = {t: i for i, t in enumerate(tag_vals)}
idx2tag = {i: t for t, i in tag2idx.items()}

Skipping row 6 due to error: [E103] Trying to set conflicting doc.ents: '(38, 57, 'COMPANY')' and '(38, 43, 'COMPANY')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 15 due to error: [E103] Trying to set conflicting doc.ents: '(1803, 1820, 'SKILLS')' and '(1803, 1808, 'COMPANY')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 18 due to error: [E103] Trying to set conflicting doc.ents: '(707, 711, 'LOC')' and '(677, 718, 'CLG')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 20 due to error: [E103] Trying to set conflicting doc.ents: '(1417, 1422, 'COMPANY')' and '(1356, 1792, 'SKILLS')'. A token 

In [104]:
labels = ['NAME', 'CLG', 'DEG', 'GRADYEAR', 'YOE', 'COMPANY', 'DESIG', 'SKILLS', 'LOC', 'EMAIL']
tags = ['O'] + [f"{prefix}-{label}" for label in labels for prefix in ['B', 'I', 'L', 'U'] if not (prefix == 'I' and label == 'CLG')] + ['[CLS]', '[SEP]', 'X']
tag2idx = {tag: idx for idx, tag in enumerate(tags)}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
print("Number of tags:", len(tag2idx))  # Should be 43

Number of tags: 43


In [105]:
from pytorch_pretrained_bert import BertTokenizer, BertForTokenClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = BertForTokenClassification.from_pretrained('/kaggle/working/bert_model/', num_labels=len(tag2idx))
bert_model = bert_model.to(device)

In [106]:
import pandas as pd
import spacy
from spacy.training import offsets_to_biluo_tags

nlp = spacy.load('en_core_web_lg')

def get_train_data(df):
    sentences = []
    tags = []
    entity_dict = {
        'Name': 'NAME',
        'College Name': 'CLG',
        'Degree': 'DEG',
        'Graduation Year': 'GRADYEAR',
        'Years of Experience': 'YOE',
        'Companies worked at': 'COMPANY',
        'Designation': 'DESIG',
        'Skills': 'SKILLS',
        'Location': 'LOC',
        'Email Address': 'EMAIL'
    }
    
    for i in range(len(df)):
        text = df['content'][i]
        annotations = df['annotation'][i] if 'annotation' in df.columns else []
        
        entities = []
        if annotations:
            for ann in annotations:
                if isinstance(ann, dict) and 'points' in ann and 'label' in ann:
                    label = ann['label'][0] if ann['label'] else None
                    if label in entity_dict:
                        for point in ann['points']:
                            start = point['start']
                            end = point['end']
                            entities.append((start, end, entity_dict[label]))
        
        doc = nlp(text)
        try:
            biluo_tags = offsets_to_biluo_tags(doc, entities)
            sentences.append([token.text for token in doc])
            tags.append(biluo_tags)
        except ValueError as e:
            print(f"Skipping row {i} due to error: {e}")
            continue
    
    return sentences, tags

df = pd.read_json('/kaggle/input/bertdata/data/Resumes.json', lines=True)
sentences, tags = get_train_data(df)
tag_vals = set(['X', '[CLS]', '[SEP]'])
for tag_list in tags:
    tag_vals.update(tag_list)
print("Training tags:", tag_vals)
print("Number of training tags:", len(tag_vals))

tag2idx = {t: i for i, t in enumerate(tag_vals)}
idx2tag = {i: t for t, i in tag2idx.items()}

Skipping row 6 due to error: [E103] Trying to set conflicting doc.ents: '(38, 57, 'COMPANY')' and '(38, 43, 'COMPANY')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 15 due to error: [E103] Trying to set conflicting doc.ents: '(1803, 1820, 'SKILLS')' and '(1803, 1808, 'COMPANY')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 18 due to error: [E103] Trying to set conflicting doc.ents: '(707, 711, 'LOC')' and '(677, 718, 'CLG')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Skipping row 20 due to error: [E103] Trying to set conflicting doc.ents: '(1417, 1422, 'COMPANY')' and '(1356, 1792, 'SKILLS')'. A token 

In [107]:
print("Columns in DataFrame:", df.columns)
print("Sample row:", df.iloc[0].to_dict())

Columns in DataFrame: Index(['content', 'annotation', 'extras'], dtype='object')
Sample row: {'content': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n1

In [108]:
expected_tags = set(['O'] + [f"{prefix}-{label}" for label in ['NAME', 'CLG', 'DEG', 'GRADYEAR', 'YOE', 'COMPANY', 'DESIG', 'SKILLS', 'LOC', 'EMAIL'] for prefix in ['B', 'I', 'L', 'U']] + ['[CLS]', '[SEP]', 'X'])
missing_tag = expected_tags - tag_vals
print("Missing tag:", missing_tag)

Missing tag: {'B-NAME', 'I-NAME', 'U-NAME', 'I-GRADYEAR', 'B-GRADYEAR', 'L-NAME', 'L-GRADYEAR'}
